In [ ]:
import csv
import openpyxl
import statistics
from collections import defaultdict
import random

anonymize_applicants = True

candidates = defaultdict(lambda: list())
reviewers = {}
with open('nominasjon.csv', 'r', encoding='utf-8') as file:
	file_reader = csv.reader(file, delimiter=',', quotechar='"')
	# Skip headers
	next(file_reader, None)
	for row in file_reader:
		name = f'{row[2]} {row[3]}'
		application_id = row[11]
		reviewer = f'{row[0]} {row[1]}'
		candidates[(name, application_id)].append((reviewer, [int(x) if x != '' else 0 for x in row[5:11]]))
		score = sum(int(x) if x != '' else 0 for x in row[5:11])
		if reviewer not in reviewers:
			reviewers[reviewer] = []
		
		if score > 0:
			reviewers[reviewer].append((name, score))

if anonymize_applicants:
	name_list = open('navn.txt', 'r', encoding='utf-8').readlines()
	anonymous_names = {}
	numbers_drawn = set()
	for candidate_name, _ in candidates:
		random_number = random.randint(0, len(name_list)-1)
		while random_number in numbers_drawn:
			random_number = random.randint(0, len(name_list)-1)
		numbers_drawn.add(random_number)
		anonymous_names[candidate_name] = name_list[random_number].capitalize()

workbook = openpyxl.Workbook()

sheet = workbook.active
sheet.title = 'Evalueringer'

reviewer_names = sorted(list(reviewer for reviewer in reviewers), key=lambda r: r.strip().lower())
reviewer_to_letter = {reviewer: chr(68 + index) for index, reviewer in enumerate(reviewer_names)}
average_letter = chr(68 + len(reviewers))
applicant_means = {}
for candidate in candidates:
	mean = 0
	no_reviewers = 0
	for reviewer, score in candidates[candidate]:
		if all(x == 0 for x in score):
			continue
		no_reviewers += 1
		mean += sum(score)
	if no_reviewers != 0:
		mean = mean / no_reviewers
	applicant_means[candidate] = mean
	
applicants = sorted([candidate for candidate in candidates], key=lambda c: applicant_means[c], reverse=True)

headers = ['Søknadsnummer', 'Søkernavn', 'Kriterium'] + reviewer_names + ['Gjennomsnitt']
sheet.freeze_panes = sheet[f'{chr(65+len(headers))}2']
criteria = ['Nysgjerrighet og lærelyst', 'Engasjement for egne og UWCs verdier', 'Sosial kompetanse', 'Robusthet og selvstendighet', 'Ansvarsbevissthet og integritet', 'Skoleprestasjoner', 'Total']

blue_color = openpyxl.styles.colors.Color(rgb='DEEAF6')
blue_fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=blue_color)

red_color = openpyxl.styles.colors.Color(rgb='FFCC99')
red_fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=red_color, bgColor=red_color)

green_color = openpyxl.styles.colors.Color(rgb='CCFFCC')
green_fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=green_color, bgColor=green_color)

applicant_totals_rows = {}

for index, header in enumerate(headers):
	sheet[f'{chr(65+index)}1'].value = header
	sheet[f'{chr(65+index)}1'].font = openpyxl.styles.Font(bold=True)

for index, applicant in enumerate(applicants):
	applicant_name, application_id = applicant
	
	sheet.merge_cells(start_row=2+(index*7), end_row=8+(index*7), start_column=1, end_column=1)
	sheet.merge_cells(start_row=2+(index*7), end_row=8+(index*7), start_column=2, end_column=2)

	for col_counter in range(len(reviewers) + 2):
		letter = chr(67+col_counter)
		sheet[f'{letter}{8+index*7}'].font = openpyxl.styles.Font(bold=True)
		sheet[f'{letter}{8+index*7}'].fill = blue_fill

	sheet[f'A{2+index*7}'].value = application_id
	sheet[f'A{2+index*7}'].alignment = openpyxl.styles.Alignment(vertical='center', horizontal='center')
	sheet[f'A{2+index*7}'].fill = blue_fill
	sheet[f'B{2+index*7}'].value = anonymous_names[applicant_name] if anonymize_applicants else applicant_name
	sheet[f'B{2+index*7}'].alignment = openpyxl.styles.Alignment(vertical='center', horizontal='center')
	sheet[f'B{2+index*7}'].fill = blue_fill

	any_non_zero = False
	for reviewer, scores in candidates[applicant]:
		letter = reviewer_to_letter[reviewer]
		non_zero_scores = False
		for score_index, score in enumerate(scores):
			sheet[f'{letter}{2+index*7+score_index}'].value = score if score != 0 else 'X'
			non_zero_scores = non_zero_scores or score > 0
		if non_zero_scores:
			any_non_zero = True
			sheet[f'{letter}{8+index*7}'].value = f'=SUM({letter}{2+index*7}:{letter}{7+index*7})'

	
	for criterion_index, criterion in enumerate(criteria):
		criterion_row = 2+index*7+criterion_index
		sheet[f'C{2+index*7+criterion_index}'].value = criterion
		sheet[f'C{2+index*7+criterion_index}'].fill = blue_fill
		if any_non_zero:
			sheet[f'{average_letter}{criterion_row}'].value = f'=AVERAGE(D{criterion_row}:{reviewer_to_letter[reviewer_names[-1]]}{criterion_row})'
			sheet[f'{average_letter}{criterion_row}'].number_format = '0.00'

	applicant_totals_rows[applicant] = 8+index*7

sheet = workbook.create_sheet('Statistikk over komiteen')
reviewer_distributions = {}
for reviewer in reviewers:
	if len(reviewers[reviewer]) > 0:
		mean = statistics.mean(score for applicant, score in reviewers[reviewer])
	else:
		mean = 0
	if len(reviewers[reviewer]) > 1:
		standard_deviation = statistics.stdev(score for applicant, score in reviewers[reviewer])
	else:
		standard_deviation = 0
	reviewer_distributions[reviewer] = (mean, standard_deviation)

reviewer_sorting = sorted([reviewer for reviewer in reviewers if len(reviewers[reviewer]) > 0], key=lambda r: reviewer_distributions[r][0], reverse=True)

sheet['A1'].value = 'Komitémedlem'
sheet['B1'].value = 'Gjennomsnittlig evaluering'
sheet['C1'].value = 'Standardavvik'

sheet['A1'].font = openpyxl.styles.Font(bold=True)
sheet['B1'].font = openpyxl.styles.Font(bold=True)
sheet['C1'].font = openpyxl.styles.Font(bold=True)

for index, reviewer in enumerate(reviewer_sorting):
	letter = reviewer_to_letter[reviewer]
	extent = f'Evalueringer!{letter}2:Evalueringer!{letter}2000'
	selection = f'IF((MOD(ROW({extent}), 7)=1)*({extent}<>""), {extent})'
	
	sheet[f'A{2+index}'].value = reviewer
	sheet[f'B{2+index}'] = openpyxl.worksheet.formula.ArrayFormula(f'B{2+index}:B{2+index}', f'=AVERAGE({selection})')
	sheet[f'C{2+index}'] = openpyxl.worksheet.formula.ArrayFormula(f'C{2+index}:C{2+index}', f'=STDEV({selection})')

	sheet[f'B{2+index}'].number_format = '0.00'
	sheet[f'C{2+index}'].number_format = '0.00'

final_row = 2+len(reviewer_sorting)
sheet[f'A{final_row}'].value = 'Total'
sheet[f'A{final_row}'].font = openpyxl.styles.Font(bold=True)

letter = chr(ord(reviewer_to_letter[reviewer_names[-1]])+1)
extent = f'Evalueringer!{letter}2:Evalueringer!{letter}2000'
selection = f'IF((MOD(ROW({extent}), 7)=1)*({extent}<>""), {extent})'

sheet[f'B{final_row}'] = openpyxl.worksheet.formula.ArrayFormula(f'B{final_row}:B{final_row}', f'=AVERAGE({selection})')
sheet[f'C{final_row}'] = openpyxl.worksheet.formula.ArrayFormula(f'C{final_row}:C{final_row}', f'=STDEV({selection})')

sheet[f'B{final_row}'].number_format = '0.00'
sheet[f'C{final_row}'].number_format = '0.00'

sheet[f'B{2+len(reviewer_sorting)}']

sheet = workbook.create_sheet('Uenighet')

sheet['A1'].value = 'Søker'
sheet['B1'].value = 'Rangering'
sheet['C1'].value = 'Standardavvik'

sheet['A1'].font = openpyxl.styles.Font(bold=True)
sheet['B1'].font = openpyxl.styles.Font(bold=True)
sheet['C1'].font = openpyxl.styles.Font(bold=True)

for index, reviewer in enumerate(reviewer_names):
	cell_index = f'{chr(69+index)}1'
	sheet[cell_index].value = reviewer
	sheet[cell_index].font = openpyxl.styles.Font(bold=True)

standard_deviations = []
for candidate in candidates:
	reviews = candidates[candidate]
	scores = [sum(scoring) for reviewer, scoring in reviews if sum(scoring) != 0]
	if len(scores) > 1:
		standard_deviation = statistics.stdev(scores)
	else:
		standard_deviation = 0
	standard_deviations.append((candidate, standard_deviation))
standard_deviations.sort(key=lambda e: e[1], reverse=True)

minimum_letter = reviewer_to_letter[reviewer_names[0]]
maximum_letter = reviewer_to_letter[reviewer_names[-1]]

for index, (candidate, standard_deviation) in enumerate(standard_deviations):
	name, application_id = candidate
	sheet[f'A{2+index}'].value = anonymous_names[name] if anonymize_applicants else name
	sheet[f'B{2+index}'].value = applicants.index(candidate)+1

	row = applicant_totals_rows[candidate]
	extent = f'Evalueringer!{minimum_letter}{row}:Evalueringer!{maximum_letter}{row}'

	sheet[f'C{2+index}'] = f'=STDEV({extent})'
	sheet[f'C{2+index}'].number_format = '0.00'

	for rev_index, reviewer in enumerate(reviewer_names):
		if any(reviewer == list_reviewer for list_reviewer, _ in candidates[candidate]):
			sheet[f'{chr(69+rev_index)}{2+index}'].value = f'=Evalueringer!{reviewer_to_letter[reviewer]}{row}'

sheet = workbook.create_sheet('Korreksjon')
sheet['B1'].value = 'Ukorrigert'
sheet['A2'].value = 'Rangering'
sheet['B2'].value = 'Søker'
sheet['C2'].value = 'Ukorrigert poengsum'
sheet['D2'].value = 'Korrigert poengsum'
sheet['E2'].value = 'Korrigert rangering'

sheet['B1'].fill = blue_fill
sheet['B2'].fill = blue_fill
sheet['C1'].fill = blue_fill
sheet['C2'].fill = blue_fill
sheet['D1'].fill = blue_fill
sheet['D2'].fill = blue_fill
sheet['E1'].fill = blue_fill
sheet['E2'].fill = blue_fill

sheet['B1'].font = openpyxl.styles.Font(bold=True)
sheet['A2'].font = openpyxl.styles.Font(bold=True)
sheet['B2'].font = openpyxl.styles.Font(bold=True)
sheet['C2'].font = openpyxl.styles.Font(bold=True)
sheet['D2'].font = openpyxl.styles.Font(bold=True)
sheet['E2'].font = openpyxl.styles.Font(bold=True)


sheet['G1'].value = 'Korrigert'
sheet['G2'].value = 'Søker'
sheet['H2'].value = 'Korrigert poengsum'
sheet['I2'].value = 'Ukorrigert poengsum'
sheet['J2'].value = 'Ukorrigert rangering'

sheet['G1'].fill = blue_fill
sheet['G2'].fill = blue_fill
sheet['H1'].fill = blue_fill
sheet['H2'].fill = blue_fill
sheet['I1'].fill = blue_fill
sheet['I2'].fill = blue_fill
sheet['J1'].fill = blue_fill
sheet['J2'].fill = blue_fill

sheet['G1'].font = openpyxl.styles.Font(bold=True)
sheet['G2'].font = openpyxl.styles.Font(bold=True)
sheet['H2'].font = openpyxl.styles.Font(bold=True)
sheet['I2'].font = openpyxl.styles.Font(bold=True)
sheet['J2'].font = openpyxl.styles.Font(bold=True)

corrected_applicants = []
total_mean = statistics.mean(applicant_mean for applicant_mean in applicant_means.values() if applicant_mean != 0)
total_stdev = statistics.stdev(applicant_mean for applicant_mean in applicant_means.values() if applicant_mean != 0)
for candidate in applicants:
	mean = 0
	no_reviewers = 0
	for reviewer, score in candidates[candidate]:
		if all(x == 0 for x in score):
			continue
		no_reviewers += 1

		reviewer_mean, reviewer_stdev = reviewer_distributions[reviewer]
		mean += (sum(score) - reviewer_mean) * total_stdev / reviewer_stdev + total_mean
	if no_reviewers > 1:
		mean = mean / no_reviewers
	corrected_applicants.append((candidate, mean))
corrected_applicants.sort(key=lambda e: e[1], reverse=True)
corrected_scores = {applicant: mean for applicant, mean in corrected_applicants}
corrected_applicants = [applicant for applicant, _ in corrected_applicants]

def get_corrected_formula(candidate):
	corrections = []
	for reviewer, score in candidates[candidate]:
		if all(x == 0 for x in score):
			continue
		uncorrected_letter = reviewer_to_letter[reviewer]
		uncorrected_row = applicant_totals_rows[candidate]

		reviewer_mean_cell = f'\'Statistikk over komiteen\'!B{2+reviewer_sorting.index(reviewer)}'
		reviewer_stdev_cell = f'\'Statistikk over komiteen\'!C{2+reviewer_sorting.index(reviewer)}'

		total_mean_cell = f'\'Statistikk over komiteen\'!B{final_row}'
		total_stdev_cell = f'\'Statistikk over komiteen\'!C{final_row}'

		corrections.append(f'(Evalueringer!{uncorrected_letter}{uncorrected_row} - {reviewer_mean_cell}) * {total_stdev_cell}/{reviewer_stdev_cell} + {total_mean_cell}')
	if len(corrections) > 0:
		return f'=MROUND(AVERAGE({",".join(corrections)}), 0.25)'
	else:
		return 0

for index, candidate in enumerate(applicants):
	sheet[f'A{3+index}'].value = index+1
	name, application_id = candidate
	sheet[f'B{3+index}'].value = anonymous_names[name] if anonymize_applicants else name
	total_letter = chr(ord(reviewer_to_letter[reviewer_names[-1]])+1)
	sheet[f'C{3+index}'].value = f'=Evalueringer!{total_letter}{applicant_totals_rows[candidate]}'
	sheet[f'D{3+index}'].value = get_corrected_formula(candidate)
	sheet[f'E{3+index}'].value = corrected_applicants.index(candidate)+1

	#sheet[f'B{3+index}'].fill = blue_fill
	#sheet[f'C{3+index}'].fill = blue_fill
		

for index, candidate in enumerate(corrected_applicants):
	name, application_id = candidate
	sheet[f'G{3+index}'].value = anonymous_names[name] if anonymize_applicants else name
	sheet[f'J{3+index}'].value = applicants.index(candidate)+1

	sheet[f'H{3+index}'].value = get_corrected_formula(candidate)
	sheet[f'H{3+index}'].number_format = '0.00'
	total_letter = chr(ord(reviewer_to_letter[reviewer_names[-1]])+1)
	sheet[f'I{3+index}'].value = f'=Evalueringer!{total_letter}{applicant_totals_rows[candidate]}'

	#sheet[f'E{3+index}'].fill = blue_fill
	#sheet[f'F{3+index}'].fill = blue_fill
	#sheet[f'G{3+index}'].fill = blue_fill

sheet['L1'].value = 'Antall til intervju'
sheet['L1'].font = openpyxl.styles.Font(bold=True)
sheet['L2'].value = 70


# Do some conditional formatting
uncorrected_range = '$B$3:INDIRECT(CONCATENATE("B", ($L$2+2)))'
corrected_range = '$G$3:INDIRECT(CONCATENATE("G", ($L$2+2)))'

for i in range(len(applicants)):
	is_removed = f'COUNTIF({corrected_range}, $B{3+i})=0'
	is_added = f'COUNTIF({corrected_range}, $B{3+i})=1'
	sheet[f'P{3+i}'].value = f'=IF({3+i}>($L$2 + 2), {is_added}, {is_removed})'

for i in range(len(applicants)):
	is_added = f'COUNTIF({uncorrected_range}, $G{3+i})=0'
	is_removed = f'COUNTIF({uncorrected_range}, $G{3+i})=1'
	sheet[f'Q{3+i}'].value = f'=IF({3+i}>($L$2 + 2), {is_removed}, {is_added})'

red_dxf = openpyxl.styles.differential.DifferentialStyle(fill=red_fill)
green_dxf = openpyxl.styles.differential.DifferentialStyle(fill=green_fill)
rule = openpyxl.formatting.Rule(type='expression', dxf=red_dxf, stopIfTrue=False)
rule.formula = [f'AND(ROW(B3) <= ($L$2 +2), $P3)']
sheet.conditional_formatting.add(f'B3:E{2+len(applicants)}', rule)

rule = openpyxl.formatting.Rule(type='expression', dxf=green_dxf, stopIfTrue=False)
rule.formula = [f'AND(ROW(B3) > ($L$2 + 2), $P3)']
sheet.conditional_formatting.add(f'B3:E{2+len(applicants)}', rule)

rule = openpyxl.formatting.Rule(type='expression', dxf=green_dxf, stopIfTrue=False)
rule.formula = [f'AND(ROW(G3) <= ($L$2 + 2), $Q3)']
sheet.conditional_formatting.add(f'G3:J{2+len(applicants)}', rule)

rule = openpyxl.formatting.Rule(type='expression', dxf=red_dxf, stopIfTrue=False)
rule.formula = [f'AND(ROW(G3) > ($L$2 + 2), $Q3)']
sheet.conditional_formatting.add(f'G3:J{2+len(applicants)}', rule)

#sheet.freeze_panes = sheet['M3']

# Adjust column widths and add cell borders
border_side = openpyxl.styles.borders.Side(style='thin')
border_style = openpyxl.styles.borders.Border(left=border_side, top=border_side, right=border_side, bottom=border_side)
for sheet in workbook.worksheets:
	for column in sheet.columns:
		width = 0
		letter = column[0].column_letter
	
		for cell in column:
			if not isinstance(cell.value, openpyxl.worksheet.formula.ArrayFormula) and not (isinstance(cell.value, str) and cell.value.startswith('=')):
				width = max(width, len(str(cell.value)))
			cell.border = border_style
	
		width += 4
	
		if sheet.title == 'Evalueringer' and letter in reviewer_to_letter.values() or sheet.title == 'Uenighet' and ord(letter) > 68:
			width = 4
	
		sheet.column_dimensions[letter].width = width

workbook.save('evalueringstabell.xlsx')

In [ ]:
import networkx as nx

pairs = []
copeland = defaultdict(lambda: int(0))
for i in range(len(applicants)):
	for j in range(i+1, len(applicants)):
		candidate_one = applicants[i]
		candidate_two = applicants[j]

		candidate_one_reviews = {reviewer: sum(score) for reviewer, score in candidates[candidate_one] if sum(score) != 0}
		candidate_two_reviews = {reviewer: sum(score) for reviewer, score in candidates[candidate_two] if sum(score) != 0}

		candidate_one_reviewers = set(candidate_one_reviews.keys())
		candidate_two_reviewers = set(candidate_two_reviews.keys())

		common_reviewers = candidate_one_reviewers.intersection(candidate_two_reviewers)
		candidate_one_preference = 0
		candidate_two_preference = 0

		for reviewer in common_reviewers:
			_, rev_stdev = reviewer_distributions[reviewer]
			preference = (candidate_one_reviews[reviewer] - candidate_two_reviews[reviewer]) * total_stdev / rev_stdev
			if preference > 0:
				candidate_one_preference += preference
			else:
				candidate_two_preference -= preference

		if candidate_one_preference > candidate_two_preference:
			pairs.append((candidate_one, candidate_two, candidate_one_preference, candidate_two_preference))
			copeland[candidate_one] += 1
		elif candidate_two_preference > candidate_one_preference:
			pairs.append((candidate_two, candidate_one, candidate_two_preference, candidate_one_preference))
			copeland[candidate_two] += 1
		else:
			copeland[candidate_one] += 1
			copeland[candidate_two] += 1

pairs.sort(key=lambda e: e[2] - e[3], reverse=True)

ranking_graph = nx.DiGraph()
ranking_graph.add_nodes_from(applicants)

cycles = 0
for index, (winner, loser, _, _) in enumerate(pairs):
	print(f'\rHandling pair {index+1} of {len(pairs)}. There are {cycles} cycles', end='')
	ranking_graph.add_edge(winner, loser)
	try:
		nx.find_cycle(ranking_graph)
		cycles += 1
		ranking_graph.remove_edge(winner, loser)
	except nx.NetworkXNoCycle:
		pass

In [ ]:
ranking = [entry for entry in nx.topological_sort(ranking_graph)]
uncorrected = set(applicants[:70])
corrected = set(ranking[:70])

print(f'Dropped out:')
for candidate in (uncorrected - corrected):
	name, _ = candidate
	print(f'\t{name}, original rank {applicants.index(candidate)}')
print('Added in:')
for candidate in (corrected - uncorrected):
	name, _ = candidate
	print(f'\t{name}, original rank {applicants.index(candidate)}')

ranking

In [ ]:
copeland_ranking = sorted([(candidate, copeland[candidate]) for candidate in copeland], key=lambda e: e[1], reverse=True)
for index, (candidate, score) in enumerate(copeland_ranking):
	print(f'{index+1}: {candidate[0]} {score}')